<a href="https://colab.research.google.com/github/Pragna-Teja-Durishetti/BCStasks/blob/main/DanceOfPlanets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Gravitational constant
G = 6.67430e-11

# User-defined values
m1 = 500
m2 = 20
x1_0 = np.array([0.0, 0.0])  # Initial position of m1
x2_0 = np.array([100.0, 0.0])  # Initial position of m2
v1_0 = np.array([0.0, 0.0])  # Initial velocity of m1
v2_0 = np.array([0.0, 5.0])  # Initial velocity of m2
t_predict =  1000  # Predict position after 1000 seconds


In [ ]:
!pip install scipy

In [ ]:
from scipy.integrate import solve_ivp

def two_body_derivatives(t, y): #The function two_body_derivatives computes the derivatives of the state vector y (positions and velocities) by calculating gravitational accelerations (a1, a2) from the mutual force between the bodies, scaled by their masses (m1, m2) and the gravitational constant G.
    x1 = y[:2]
    x2 = y[2:4]
    v1 = y[4:6]
    v2 = y[6:8]
    r = np.linalg.norm(x2 - x1)
    a1 = G * m2 * (x2 - x1) / r**3
    a2 = G * m1 * (x1 - x2) / r**3
    return np.concatenate([v1, v2, a1, a2])

initial_state = np.concatenate([x1_0, x2_0, v1_0, v2_0])
time_span = (0, t_predict)
time_eval = np.linspace(*time_span, 1000)
sol = solve_ivp(two_body_derivatives, time_span, initial_state, t_eval=time_eval) #Using solve_ivp from SciPy, it numerically integrates these equations over a specified time span (time_span), returning the positions and velocities at 1000 evenly spaced time points (time_eval).

X_data = sol.t.reshape(-1, 1)
y_data = sol.y.T[:, [0,1,2,3]]  # x1 and x2 positions only


In [ ]:

# Train-test split
split_idx = int(0.8 * len(X_data))
X_train, X_test = X_data[:split_idx], X_data[split_idx:]
y_train, y_test = y_data[:split_idx], y_data[split_idx:]


In [ ]:
# --- Model Definition Cell ---
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 128), # Input layer
            nn.Tanh(),# 3 Hidden layers with Tanh activation for smooth nonlinearity
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 4) # Output LAYER : nn.Linear(128, 4) (predicts 4 values: likely the [x1, y1, x2, y2] positions of the two bodies)
        )

    def forward(self, t):
        return self.net(t)

model = Net()
criterion = nn.MSELoss()# Mean Squared Error
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) #Adam (adaptive learning rate, default lr=1e-3) for gradient descent

# Convert data to torch tensors
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32)

# Training loop
for epoch in range(10000):
    optimizer.zero_grad()
    outputs = model(X_train_torch)
    loss = criterion(outputs, y_train_torch) #Compares predictions (outputs) to true positions (y_train_torch) using MSE
    loss.backward()#loss.backward() computes gradients
    optimizer.step()#optimizer.step() updates weights
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}") #Prints loss every 1000 epochs to monitor convergence


Epoch 0, Loss: 1336085.875000
Epoch 1000, Loss: 1203912.500000
Epoch 2000, Loss: 1091277.875000
Epoch 3000, Loss: 987566.500000
Epoch 4000, Loss: 891553.625000
Epoch 5000, Loss: 802533.500000
Epoch 6000, Loss: 720019.687500
Epoch 7000, Loss: 643636.937500
Epoch 8000, Loss: 573084.312500
Epoch 9000, Loss: 508091.312500


In [ ]:

# --- Evaluation Criteria Cell ---
from sklearn.metrics import mean_absolute_error

# Prediction on test data

X_test_torch = torch.tensor(X_test, dtype=torch.float32)#Converts test input (time values) to a PyTorch tensor
preds = model(X_test_torch).detach().numpy()# Runs the model on test data and converts the output tensor to a NumPy array
mae_position = mean_absolute_error(y_test, preds) # Computes MAE between true positions (y_test) and predictions (preds)
print(f"MAE on position prediction: {mae_position:.6f} m")


MAE on position prediction: 820.774889 m


In [ ]:

# --- Final Model Performance Cell ---
x1_final, x2_final = model(torch.tensor([[t_predict]], dtype=torch.float32)).detach().numpy().reshape(2,2) # Feeds the time t_predict into the trained neural network then converts the output tensor to a NumPy array and detaches it from the computation graph and Reshapes the output into two 2D coordinates
print(f"Predicted position at t={t_predict} s")
print(f"x1: {x1_final}, x2: {x2_final}")


Predicted position at t=1000 s
x1: [ 0.00116542 -0.00146379], x2: [  99.98431 1218.921  ]
